In [2]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
import os
from IPython.display import display, Markdown

load_dotenv(override=True)

True

In [11]:
import os
import nest_asyncio
from agents.mcp import MCPServerStdio

nest_asyncio.apply()

async def list_tools():
    script_path = os.path.abspath("../mcp/server.py")
    params = {"command": "uv", "args": ["run", script_path]}

    async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
        tools = await server.list_tools()

        print("\n✅ Available MCP Tools:")
        for t in tools:
            # Tool object → use attributes instead of dict access
            print(f"- {t.name}: {t.description}")

        return tools

await list_tools()



✅ Available MCP Tools:
- run_axe_audit: 
    Run axe-core on the page and return the full axe result (violations, passes, incomplete, etc).
    This is the primary automated scanner and is required for broad WCAG coverage.
    
- check_headings: 
- check_language: 
- check_images: 
- check_forms: 
- check_aria: 
- check_keyboard: 
- check_videos: 
- check_contrast: 
- check_semantics: 
- full_audit: 
    Runs axe + all custom checks and returns a comprehensive report with suggested remediation hints.
    


[Tool(name='run_axe_audit', description='\n    Run axe-core on the page and return the full axe result (violations, passes, incomplete, etc).\n    This is the primary automated scanner and is required for broad WCAG coverage.\n    ', inputSchema={'properties': {'url': {'title': 'Url', 'type': 'string'}, 'timeout': {'default': 30000, 'title': 'Timeout', 'type': 'integer'}}, 'required': ['url'], 'title': 'run_axe_auditArguments', 'type': 'object'}, annotations=None),
 Tool(name='check_headings', description='', inputSchema={'properties': {'url': {'title': 'Url', 'type': 'string'}}, 'required': ['url'], 'title': 'check_headingsArguments', 'type': 'object'}, annotations=None),
 Tool(name='check_language', description='', inputSchema={'properties': {'url': {'title': 'Url', 'type': 'string'}}, 'required': ['url'], 'title': 'check_languageArguments', 'type': 'object'}, annotations=None),
 Tool(name='check_images', description='', inputSchema={'properties': {'url': {'title': 'Url', 'type': 'st

In [14]:
import os
import nest_asyncio
import asyncio
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
from IPython.display import Markdown, display

# Allow nested async in Jupyter
nest_asyncio.apply()

async def run_agent():
    # ---------------------------------------------------------------------
    # Setup
    # ---------------------------------------------------------------------
    script_path = os.path.abspath("../mcp/server.py")
    if not os.path.exists(script_path):
        raise FileNotFoundError(f"MCP server script not found at: {script_path}")

    print(f"🚀 Launching Accessibility MCP server at: {script_path}")

    params = {"command": "uv", "args": ["run", script_path]}

    instructions = (
        "You are an AI assistant specialized in ADA and WCAG 2.2 compliance. "
        "Your job is to evaluate webpages using the available MCP tools. "
        "Provide a compliance rating, identify accessibility issues, "
        "and suggest detailed fixes in Markdown format."
    )

    request = "Audit oauthapp.azurewebsites.net for ADA and WCAG compliance and summarize findings in Markdown."

    model = "gpt-4.1-mini"

    # ---------------------------------------------------------------------
    # Run the agent with MCP server
    # ---------------------------------------------------------------------
    try:
        async with MCPServerStdio(params=params, client_session_timeout_seconds=90) as accessibility_server:
            print("✅ Connected to MCP server. Listing available tools...")
            tools = await accessibility_server.list_tools()
            print(f"🔧 {len(tools)} tools loaded:")
            for tool in tools:
                print(f"   - {tool.name}: {tool.description}")

            agent = Agent(
                name="accessibility_agent",
                instructions=instructions,
                model=model,
                mcp_servers=[accessibility_server],
            )

            with trace("accessibility_agent"):
                result = await Runner.run(agent, request)

            # -----------------------------------------------------------------
            # Display or fallback if model output is empty
            # -----------------------------------------------------------------
            if not result or not result.final_output:
                display(Markdown("⚠️ **No audit results returned.** Please check the MCP server logs."))
            else:
                display(Markdown(result.final_output))

    except asyncio.TimeoutError:
        print("⏱️ MCP server timed out. Try increasing `client_session_timeout_seconds`.")
    except Exception as e:
        print(f"❌ Unexpected error: {e}")

# Run inside notebook
await run_agent()


🚀 Launching Accessibility MCP server at: /home/azureuser/ws/agenticai/projects/accessibility/mcp/server.py
✅ Connected to MCP server. Listing available tools...
🔧 11 tools loaded:
   - run_axe_audit: 
    Run axe-core on the page and return the full axe result (violations, passes, incomplete, etc).
    This is the primary automated scanner and is required for broad WCAG coverage.
    
   - check_headings: 
   - check_language: 
   - check_images: 
   - check_forms: 
   - check_aria: 
   - check_keyboard: 
   - check_videos: 
   - check_contrast: 
   - check_semantics: 
   - full_audit: 
    Runs axe + all custom checks and returns a comprehensive report with suggested remediation hints.
    


# Accessibility Audit Report for oauthapp.azurewebsites.net

## Compliance Summary
- **WCAG 2.1 Level A and AA Violations:** 0 detected
- **Color Contrast Issues:** None found (all text passes AA contrast ratio criteria)
- **Missing Alt Attributes on Images:** None
- **Forms Without Labels:** None (no input forms detected)
- **ARIA Issues:** Some role="img" SVG elements missing accessible names, and one duplicate id attribute.
- **Keyboard Accessibility:** No critical issues, 24 focusable elements with a logical tab order.
- **Headings:** Proper use of headings with a single H1 and logically structured H2s.
- **Language:** Valid lang attribute (`en`) specified on the HTML element.
- **Videos:** None present, so no caption requirements.
- **Landmarks:** Site makes use of landmarks: header, main, nav, footer, aside.

## Detailed Findings

### Positive Findings
- Document has meaningful and properly structured headings (1 H1, followed by H2s).
- All images have valid alt text, providing alternative descriptions.
- Color contrast ratios meet or exceed WCAG 2.1 AA standards (4.5:1 for normal text, 3:1 for large text).
- The document contains a valid `<title>` element for proper page identification.
- The document's `<html>` element includes a valid `lang="en"` attribute.
- Navigation and structural landmarks (header, nav, main, footer, aside) are present for assistive technology.
- Keyboard focus order is logical and functional.
- No form inputs lacking labels (no input fields found).
- No use of deprecated or disallowed elements like `<blink>` or `<marquee>`.
- No meta-refresh or zoom disabling issues detected.

### Accessibility Issues

1. **SVG Elements with `role="img"` Missing Accessible Names:**
   - Multiple Font Awesome SVG icons used as images have the `role="img"` attribute but lack accessible names (`aria-label` or `<title>`).
   - This can be problematic for screen reader users because these images do not have descriptive alternative text.
   - Example elements affected include icons like plus, book, code-branch, key, sign-in-alt, unlock-alt, users, bars, etc.

2. **Duplicate ID Attribute:**
   - The ID `docIcon` is present on multiple SVG elements, which violates the uniqueness requirement for IDs in HTML.
   - Duplicate IDs can cause issues in assistive technologies and scripting.

3. **Missing Skip Link:**
   - No "skip to content" link was detected. While not required, it is a strong recommended practice to help keyboard and screen reader users bypass repetitive navigation.

## Recommendations

### Fix SVG Accessibility

- Add accessible names to SVG icons with `role="img"`.
- Methods:
  - Add a `<title>` element inside the SVG describing the icon (e.g., `<title>Expand</title>`).
  - Or add `aria-label="Description of icon"` attribute directly on the SVG.
- Ensure that decorative SVGs use `aria-hidden="true"` and no `role="img"`, if they convey no meaningful content.

### Fix Duplicate IDs

- Rename one of the duplicated `docIcon` IDs to a unique ID to meet HTML specifications.
- Confirm all IDs on the page are unique.

### Add a Skip Link

- Add a visually hidden but keyboard-accessible "Skip to main content" link at the top of the page.
- Example:
  ```html
  <a href="#main-content" class="skip-link">Skip to main content</a>
  ```
- Ensure the main content container has an `id="main-content"`.

---

## Overall Compliance Rating: **High**

The page demonstrates strong compliance with WCAG 2.1 Level A and AA criteria. Only minor ARIA and duplicate ID issues were found, which are relatively simple to fix. Addressing these will further improve accessibility and screen reader experience.

---

If you want, I can provide sample code snippets or detailed instructions for the fixes.

In [18]:
import os
import nest_asyncio
import asyncio
from datetime import datetime
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
from IPython.display import Markdown, display, HTML

nest_asyncio.apply()

async def run_site_accessibility_dashboard(base_url: str):
    # ------------------- Setup MCP Server -------------------
    script_path = os.path.abspath("../mcp/server.py")
    if not os.path.exists(script_path):
        raise FileNotFoundError(f"MCP server script not found at: {script_path}")

    print(f"🚀 Launching Accessibility MCP server at: {script_path}")
    params = {"command": "uv", "args": ["run", script_path]}
    model = "gpt-4.1-mini"

    try:
        async with MCPServerStdio(params=params, client_session_timeout_seconds=120) as accessibility_server:
            print("✅ Connected to MCP server.")

            # ------------------- Fetch sitemap -------------------
            import aiohttp
            from xml.etree import ElementTree as ET
            urls_to_audit = [base_url]
            sitemap_url = base_url.rstrip("/") + "/sitemap.xml"
            try:
                async with aiohttp.ClientSession() as session:
                    async with session.get(sitemap_url) as resp:
                        if resp.status == 200:
                            xml_text = await resp.text()
                            root = ET.fromstring(xml_text)
                            urls_to_audit = [elem.text for elem in root.findall(".//{*}loc")]
            except Exception as e:
                print(f"⚠️ Sitemap fetch error: {e}")
            
            print(f"🔧 Pages to audit: {len(urls_to_audit)}")

            # ------------------- Run Accessibility Audit per Page -------------------
            audit_results = {}
            audit_instructions = (
                "You are an AI assistant specialized in ADA/WCAG 2.2 compliance. "
                "Audit a webpage and produce a Markdown report including all rules (Pass/Fail) "
                "with columns: Level (A, AA, AAA), Rule, Pass/Fail, Reason, Recommendation."
            )

            for url in urls_to_audit:
                print(f"📄 Auditing: {url}")
                audit_agent = Agent(
                    name="accessibility_agent",
                    instructions=audit_instructions,
                    model=model,
                    mcp_servers=[accessibility_server],
                )
                with trace(f"audit_{url}"):
                    result = await Runner.run(audit_agent, f"Audit {url} for ADA/WCAG compliance.")
                markdown_output = result.final_output if result and result.final_output else f"⚠️ No results for {url}"
                audit_results[url] = markdown_output
                display(Markdown(f"### Audit for {url}"))
                display(Markdown(markdown_output))

            # ------------------- HTML Dashboard -------------------
            html_instructions = (
                "You are an AI assistant that converts multiple Markdown accessibility audit reports "
                "into a single responsive HTML dashboard. "
                "Requirements: \n"
                "- Hero banner with site name\n"
                "- Filters: All / Passed / Failed / Warnings\n"
                "- Clicking a filter updates the table dynamically\n"
                "- Table columns: Level, Rule, Pass/Fail, Reason, Recommendation\n"
                "- TailwindCSS styling\n"
                "- Include a summary row with total Pass/Fail counts per page"
            )

            html_agent = Agent(
                name="html_dashboard_agent",
                instructions=html_instructions,
                model=model
            )

            html_request = "Generate a site-wide accessibility dashboard from the following Markdown reports:\n\n"
            for url, md in audit_results.items():
                html_request += f"Page: {url}\n{md}\n\n"

            with trace("html_dashboard_agent"):
                html_result = await Runner.run(html_agent, html_request)

            html_output = html_result.final_output if html_result and html_result.final_output else "<p>⚠️ HTML dashboard generation failed.</p>"
            display(HTML(html_output))

            # ------------------- Save HTML -------------------
            output_dir = os.path.abspath("output")
            os.makedirs(output_dir, exist_ok=True)
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            output_file = os.path.join(output_dir, f"accessibility_dashboard_{timestamp}.html")
            with open(output_file, "w", encoding="utf-8") as f:
                f.write(html_output)

            print(f"✅ Site-wide HTML dashboard saved to: {output_file}")

    except asyncio.TimeoutError:
        print("⏱️ MCP server timed out. Increase `client_session_timeout_seconds` if needed.")
    except Exception as e:
        print(f"❌ Unexpected error: {e}")

# Run dashboard
await run_site_accessibility_dashboard("https://oauthapp.azurewebsites.net")


🚀 Launching Accessibility MCP server at: /home/azureuser/ws/agenticai/projects/accessibility/mcp/server.py
✅ Connected to MCP server.
🔧 Pages to audit: 5
📄 Auditing: https://oauthapp.azurewebsites.net/


### Audit for https://oauthapp.azurewebsites.net/

# Accessibility Audit Report for https://oauthapp.azurewebsites.net/

| Level | Rule                          | Pass/Fail | Reason                                                                                      | Recommendation                                                                                       |
|-------|-------------------------------|-----------|---------------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------|
| A     | html-has-lang                 | Pass      | The `<html>` element has a lang attribute set to "en".                                      | No action needed.                                                                                   |
| A     | valid-lang                   | Pass      | The lang attribute has a valid value "en".                                                 | No action needed.                                                                                   |
| A     | document-title               | Pass      | The document contains a non-empty `<title>` element.                                       | No action needed.                                                                                   |
| A     | image-alt                    | Pass      | All images have alternate text provided.                                                   | No action needed.                                                                                   |
| A     | label                       | Pass      | No form inputs requiring labels found.                                                     | No action needed.                                                                                   |
| A     | link-name                   | Pass      | All links have discernible text.                                                           | No action needed.                                                                                   |
| A     | button-name                 | Pass      | All buttons have discernible text.                                                         | No action needed.                                                                                   |
| A     | aria-hidden-body            | Pass      | No `aria-hidden="true"` on the `<body>`.                                                   | No action needed.                                                                                   |
| A     | aria-hidden-focus           | Pass      | ARIA hidden elements are not focusable and contain no focusable children.                   | No action needed.                                                                                   |
| AA    | color-contrast              | Pass      | Contrast ratios meet or exceed WCAG 2 AA minimum threshold.                                | No action needed.                                                                                   |
| A     | headings                   | Pass      | The page has a single `<h1>` and properly structured `<h2>` headings, no order problems.    | No action needed.                                                                                   |
| A     | keyboard                   | Pass      | All interactive elements are focusable and tabbable in a logical order.                     | No action needed.                                                                                   |
| A     | list                      | Pass      | Lists are correctly structured with only allowed immediate children.                        | No action needed.                                                                                   |
| A     | listitem                   | Pass      | All `<li>` elements are contained inside `<ul>` or `<ol>`.                                 | No action needed.                                                                                   |
| A     | meta-viewport              | Pass      | Meta viewport tag does not disable zooming/scaling.                                        | No action needed.                                                                                   |
| A     | nested-interactive          | Pass      | There are no nested interactive controls.                                                  | No action needed.                                                                                   |
| A     | scrollable-region-focusable | Pass      | Scrollable regions have keyboard access.                                                   | No action needed.                                                                                   |
| A     | semantics                  | Pass      | Page uses landmarks such as header, main, nav, footer, aside correctly.                     | Consider adding a "Skip to content" link for improved navigation by keyboard/screen reader users.   |

| Level | Rule                          | Pass/Fail | Reason                                                                                      | Recommendation                                                                                       |
|-------|-------------------------------|-----------|---------------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------|
| A     | aria-role-img-alt              | Fail      | Multiple `<svg>` elements with role="img" lack accessible names.                           | Provide accessible names using `aria-label` or `aria-labelledby` for all SVG images with `role="img"`.|

| Level | Rule                          | Pass/Fail | Reason                                                                                      | Recommendation                                                                                       |
|-------|-------------------------------|-----------|---------------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------|
| A     | duplicate-id                  | Fail      | The ID "docIcon" is used more than once in the DOM.                                        | Ensure all ID attributes are unique in the document to avoid conflicts during navigation and assistive technology usage.|

| Level | Rule                          | Pass/Fail | Reason                                                                                      | Recommendation                                                                                       |
|-------|-------------------------------|-----------|---------------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------|
| A     | has-skip-link                 | Fail      | The page does not contain any mechanism to skip repetitive content (such as a "Skip to content" link). | Add a "Skip to content" link as an immediate child of the body for keyboard users to bypass navigation.|

---

### Summary:
- The site meets many core WCAG 2.1 Level A and AA accessibility requirements including language declaration, image alt text, headings structure, contrast, keyboard accessibility, and ARIA usage in most areas.
- Major issues to address:
  - Provide accessible names for decorative or functional SVG images flagged with role="img".
  - Ensure all ID attributes are unique, fixing duplicate ID "docIcon".
  - Add a skip link to improve keyboard navigation for screen reader users.

Addressing these will improve the site's accessibility and compliance with WCAG 2.1 A and AA standards.

📄 Auditing: https://oauthapp.azurewebsites.net/code


### Audit for https://oauthapp.azurewebsites.net/code

...

📄 Auditing: https://oauthapp.azurewebsites.net/password


### Audit for https://oauthapp.azurewebsites.net/password

# ADA/WCAG Compliance Audit Report for https://oauthapp.azurewebsites.net/password

| Level | Rule                              | Pass/Fail | Reason                                                                                          | Recommendation                                                                                       |
|-------|-----------------------------------|-----------|------------------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------|
| A     | list                              | Fail      | An ordered list (<ol>) contains a <pre> element as a direct child, which is not allowed.       | Ensure <ul> and <ol> elements only contain <li>, <script>, or <template> elements as direct children.|
| A     | role-img-alt                     | Fail      | Multiple <svg> icons with role="img" are missing accessible names (alt text equivalent).       | Provide accessible names for all <svg> elements with role="img", e.g., aria-label or <title> element.|
| A     | duplicate-id                    | Fail      | Duplicate ID "docIcon" found on multiple elements.                                            | Ensure all id attributes are unique on the page.                                                  |
| A     | html-has-lang                  | Pass      | The <html> element has a valid lang attribute set to "en".                                    | None needed.                                                                                    |
| A     | image-alt                      | Pass      | All <img> elements have alt attributes with valid values.                                     | None needed.                                                                                    |
| A     | forms-label                   | Pass      | No forms or form fields present that are missing labels.                                      | None needed.                                                                                    |
| A     | button-name                   | Pass      | Buttons have discernible text or accessible names.                                            | None needed.                                                                                    |
| AA    | color-contrast                | Pass      | Text and UI components meet the minimum contrast ratio required by WCAG 2.1 AA.                | None needed.                                                                                    |
| A     | keyboard                      | Pass      | Keyboard navigation works; all interactive elements are focusable in correct order.           | None needed.                                                                                    |
| A     | headings                      | Pass      | Proper heading structure is present with no order problems, but no <h1> found.               | Recommend adding a single <h1> heading for improved semantic structure and accessibility.       |
| A     | semantics-landmarks           | Pass      | Page contains multiple landmark elements like header, main, nav, and footer.                  | Consider adding a "skip to content" link to facilitate bypassing repetitive navigation.          |
| AA    | meta-viewport                | Pass      | Meta viewport tag correctly configured without disabling zoom.                                | None needed.                                                                                    |

## Summary
The page is mostly compliant with fundamental WCAG 2.1 A and AA requirements but has several accessibility issues that must be addressed:

- The key violation is improper list structure where a <pre> element exists directly inside an <ol>. This breaks semantic list requirement.
- Numerous SVG icons lack accessible names which impacts screen reader users.
- Duplicate IDs on the page violate unique ID constraints, which can cause accessibility and scripting issues.
- The page misses a primary <h1> heading which is recommended for semantic clarity and accessibility.
- Adding a skip link is recommended to allow keyboard users to bypass navigation easily.

Addressing these issues will enhance accessibility for users relying on assistive technologies and keyboard navigation.

---

For detailed guidance, refer to the WCAG 2.2 checklist and [Deque University Axe rules](https://dequeuniversity.com/rules/axe/4.8/).

Error getting response: Connection error.. (request_id: None)


❌ Unexpected error: Connection error.
